In [2]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 918.5 kB/s eta 0:00:0031m1.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 50.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 46.6 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_csv('data/COVID-19SurveyStudentResponses.csv')

df = df.dropna()
print(len(df))

df

1115


,ID,Region of residence,Age of Subject,Time spent on Online Class,Rating of Online Class experience,Medium for online class,Time spent on self study,Time spent on fitness,Time spent on sleep,Time spent on social media,Prefered social media platform,Time spent on TV,Number of meals per day,Change in your weight,Health issue during lockdown,Stress busters,Time utilized,"Do you find yourself more connected with your family, close friends , relatives ?",What you miss the most
0,R1,Delhi-NCR,21,2.0,Good,Laptop/Desktop,4.0,0.0,7.0,3.0,Linkedin,1,4,Increased,NO,Cooking,YES,YES,School/college
1,R2,Delhi-NCR,21,0.0,Excellent,Smartphone,0.0,2.0,10.0,3.0,Youtube,0,3,Decreased,NO,Scrolling through social media,YES,NO,Roaming around freely
2,R3,Delhi-NCR,20,7.0,Very poor,Laptop/Desktop,3.0,0.0,6.0,2.0,Linkedin,0,3,Remain Constant,NO,Listening to music,NO,YES,Travelling
3,R4,Delhi-NCR,20,3.0,Very poor,Smartphone,2.0,1.0,6.0,5.0,Instagram,0,3,Decreased,NO,Watching web series,NO,NO,"Friends , relatives"
4,R5,Delhi-NCR,21,3.0,Good,Laptop/Desktop,3.0,1.0,8.0,3.0,Instagram,1,4,Remain Constant,NO,Social Media,NO,NO,Travelling
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,R1191,Delhi-NCR,12,3.0,Good,Smartphone,4.0,1.0,8.0,1.0,Instagram,2,3,Decreased,NO,Dancing,YES,YES,Travelling
1178,R1192,Delhi-NCR,14,6.0,Average,Smartphone,4.0,1.0,9.0,1.0,Whatsapp,1,4,Remain Constant,NO,Listening to music,YES,YES,"Friends , relatives"
1179,R1193,Delhi-NCR,13,4.0,Average,Smartphone,0.0,0.5,8.0,3.0,Youtube,2,4,Decreased,NO,Online gaming,NO,YES,School/college
1180,R1194,Delhi-NCR,14,5.0,Excellent,Laptop/Desktop,3.5,1.0,8.0,0.5,Youtube,1,4,Remain Constant,NO,Reading books,YES,YES,School/college


In [13]:
%pip install openai tenacity python-dotenv

from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_random_exponential
from typing import List
import openai
import os

load_dotenv()

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model, **kwargs) -> List[float]:

    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")
    text = text.replace("<br/>", " ")

    response = openai.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

Note: you may need to restart the kernel to use updated packages.


In [14]:
text="great location!"
model="text-embedding-ada-002"
embedding=get_embedding(text, model)

print("embeddings length: ", len(embedding))
print(embedding)

embeddings length:  1536
[-0.014858539216220379, -0.005779752507805824, 0.002443669131025672, -0.0408770926296711, -0.01077340729534626, 0.010522114112973213, -0.007094211410731077, -0.017100851982831955, 0.020374111831188202, -0.030541837215423584, 0.005931172985583544, 0.030490288510918617, -0.006855804473161697, -0.014948747120797634, -0.0060020508244633675, -0.003252319060266018, 0.015180710703134537, -0.008672851137816906, 0.0021730451844632626, -0.01067031268030405, -0.032217126339673996, 0.012223178520798683, -0.004371864255517721, -0.035825446248054504, 0.004433076828718185, 0.006285561714321375, -0.0036340919323265553, -0.004394416231662035, -0.013466759584844112, 0.0017348922556266189, 0.016959095373749733, -0.006317778490483761, -0.03546461462974548, -0.01640496216714382, -0.009014352224767208, 0.0009439618443138897, -0.0009753735503181815, -0.021198870614171028, 0.017719419673085213, -0.006694719195365906, -0.006272674538195133, -0.01333789061754942, 0.002781949006021023, 0

In [15]:
import os 

embedding=get_embedding(text, model)
df["embedding"] = df.PreferedSocialMediaPlatform.apply(lambda x: get_embedding(x, model=model))
output_path="data/output/reviews-embeddings.csv"


if os.path.exists(output_path):
    os.remove(output_path)

df.to_csv(output_path) #test comment

AttributeError: 'DataFrame' object has no attribute 'PreferedSocialMediaPlatform'